In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from dGbyG.utils.custom_tools import rapid_process_result, rapid_linear_reg
from dGbyG.config import train_data_path

Reading the 10-fold cross validation results data of dGbyG

In [2]:
TrainingData_df = pd.read_csv(train_data_path)
is_rxn = np.array([' = ' in r for r in TrainingData_df.loc[:,'reaction']])
real_r = TrainingData_df.loc[:,'standard_dg_prime'].to_numpy().reshape(-1)
mean_std = TrainingData_df.loc[:,'std'].mean()
SEM = TrainingData_df.loc[:,'SEM'].to_numpy()

Scale = []
for n, sem in zip(TrainingData_df.loc[:,'n'], TrainingData_df.loc[:,'SEM']):
    scale = mean_std if np.isnan(sem) else (sem**2 + mean_std**2/n)**0.5
    Scale.append(scale)
Scale = np.array(Scale)

results_dir = '../data/results_data/cross_validation_results'
Loss = []
Results = []
Final_p = []
R = []
for n in range(0,100):
    name = '10_fold_cross_validation_with_random_dG_'+str(n)
    loss = np.load(os.path.join(results_dir, name+'.npy')).mean(axis=0)
    res = pd.read_csv(os.path.join(results_dir, name+'.csv'), index_col=0)
    idx = np.argmin(loss)

    Loss.append(loss)
    Results.append(res)
    Final_p.append(res.loc[:,str(idx)].to_numpy())
    R.append(res['r'].to_numpy())

Loss = np.array(Loss)
R = np.array(R)
Final_p = np.array(Final_p)

p = Final_p.mean(axis=0)
ae = np.abs(p - real_r)

Reading the cross validation results data of CC method and AF method